### Introduction


This notebook will create the context() tool for the llm agent. The context tool should return similar chunks based from querry using the faiss vector store avaible in the knowledge pack. It should also format the chunks and add them to chat history. 

In [21]:
# === 0) Imports & manifest ===
from pathlib import Path
import yaml, json
from typing import Optional, List, Dict
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langchain.schema import Document

# If you want the prebuilt ReAct agent:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# If you prefer to also expose tools directly on the LLM:
from langchain.chat_models import init_chat_model

# --- Set your pack root + manifest ---
ROOT = Path("/Users/ktejwani/Personal CS Projects/Summer 2025/Offline AI Kiosk/Offline-AI-Kiosk/first_aid_pack_demo_v2")
MANIFEST = ROOT / "manifest.yaml"

with open(MANIFEST, "r", encoding="utf-8") as f:
    manifest = yaml.safe_load(f)

# --- Resolve FAISS paths from manifest ---
faiss_dir = ROOT / manifest["precomputed_indices"]["text"]["faiss"]["dir"]

# --- Create embeddings *matching the store* ---
embed_model_name = manifest["embedding_config"]["text"]["model"]     # e.g., "granite-embedding:30m"
emb = OllamaEmbeddings(model=embed_model_name)

# --- Load FAISS store + retriever ---
vs = FAISS.load_local(str(faiss_dir), emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(search_kwargs={"k": 4})  # default k; tool will override if provided

# === 1) Helpers ===
def format_chunk(doc: Document, max_chars: int = 400) -> Dict:
    """Return a dict with compact text + key metadata for prompting & audit."""
    txt = doc.page_content.strip()
    if len(txt) > max_chars:
        txt = txt[:max_chars].rstrip() + " …"
    m = doc.metadata
    return {
        "id": m.get("chunk_id"),
        "topic_id": m.get("topic_id"),
        "file_id": m.get("file_id"),
        "locale": m.get("locale"),
        "path": m.get("path"),
        "citations": [c.get("title", "") for c in m.get("citations", [])],
        "text": txt
    }

def format_context_block(chunks: List[Dict]) -> str:
    """Human/LLM-friendly context block the agent can drop into its reasoning."""
    lines = []
    lines.append("### Retrieved Context (use only what is relevant)")
    for i, c in enumerate(chunks, 1):
        cite_str = "; ".join([t for t in c["citations"] if t]) or "—"
        head = f"[{i}] {c['topic_id']} · {c['file_id']} · {c['locale']} · {c['path']}"
        lines.append(head)
        lines.append(c["text"])
        lines.append(f"Source(s): {cite_str}")
        lines.append("")  # blank line
    return "\n".join(lines).strip()

# === 2) The @tool: context() ===
@tool
def context(
    query: str,
    k: int = 4,
    topic_id: Optional[str] = None,
    locale: Optional[str] = None
) -> dict:
    """
    Retrieve up to k relevant knowledge-pack chunks for 'query' and return a formatted
    context block + structured per-chunk data for citations. You must use this tool for any prompt that is 
    important to wellbeing or safety of user. 

    Args:
        query: Natural language question or keywords.
        k: Top-k chunks to return (default 4).
        topic_id: Optional manifest topic filter (e.g., 'bleed-control').
        locale: Optional locale filter (e.g., 'hi_en' or 'en').

    Returns:
        {
          "query": str,
          "k": int,
          "filters": {"topic_id":..., "locale":...},
          "context_block": str,     # pasteable into prompts
          "chunks": [ {id, topic_id, file_id, path, locale, citations[], text}, ... ]
        }
    """
    # Build a metadata filter if provided
    _filter = {}
    # if topic_id:
    #     _filter["topic_id"] = topic_id
    # if locale:
    #     _filter["locale"] = locale

    # Run retrieval (override k)
    local_ret = vs.as_retriever(search_kwargs={"k": k})
    hits: List[Document] = local_ret.invoke(query) if not _filter else local_ret.invoke(query, filter=_filter)

    formatted = [format_chunk(d) for d in hits]
    ctx_block = format_context_block(formatted)
    return {
        "query": query,
        "k": k,
        "filters": _filter,
        "context_block": ctx_block,
        "chunks": formatted
    }

# Keep a Tools list for whichever orchestration you choose:
TOOLS = [context]


### Testing Tool Requests

In [31]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="ollama:gpt-oss:20b",       
    temperature=0.2  # lower = more deterministic
)

llm_with_tools = llm.bind_tools(TOOLS) #llm_with_tools is a new wrapped llm
query = "What do if bleeding?"
llm_with_tools.invoke(query) 


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-27T02:47:36.848995Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13818915250, 'load_duration': 8068646875, 'prompt_eval_count': 312, 'prompt_eval_duration': 3420809500, 'eval_count': 83, 'eval_duration': 2313899708, 'model_name': 'gpt-oss:20b'}, id='run--fc1d3973-604d-4919-9250-8ba69635eb0c-0', tool_calls=[{'name': 'context', 'args': {'k': 4, 'locale': 'en', 'query': 'bleeding emergency first aid', 'topic_id': None}, 'id': '3975110a-625c-4973-979d-9aae04e2adc9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 312, 'output_tokens': 83, 'total_tokens': 395})

### Full Agent Sim

In [32]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage

#message classes on lang chain inlcude human massage ,ai message, system message, and tool message

SYSTEM_RULES = SystemMessage(content=(
    "You are a first-aid assistant. If additional facts are needed, "
    "CALL the `context` tool (do not describe it). After receiving tool output, "
    "base your answer on it and end with a 'Sources:' line listing citation titles. "
    "If no retrieved context is available, say you don't have enough info."
))


query = "What do if bleeding? What to do if snakebite?"


messages = [SYSTEM_RULES, HumanMessage(content=query)]

ai_msg = llm_with_tools.invoke(messages) #llm_with_tools looks at history(currently only 1 human message) and then builds prompt
print("AI MESSAEGE CALLS")
print(ai_msg)
print("JUST AI TOOL CALLS")
print(ai_msg.tool_calls) 
#Now we add the bots message to the chat history
messages.append(ai_msg)

AI MESSAEGE CALLS
content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-27T02:48:00.578782Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6079019834, 'load_duration': 75429792, 'prompt_eval_count': 385, 'prompt_eval_duration': 2960362417, 'eval_count': 108, 'eval_duration': 3015826583, 'model_name': 'gpt-oss:20b'} id='run--6b2de3ab-4c8b-4b5c-a3d2-08fe35457a41-0' tool_calls=[{'name': 'context', 'args': {'k': 4, 'locale': 'en', 'query': 'bleeding first aid', 'topic_id': 'bleed-control'}, 'id': '9f960820-0553-4df4-9794-3e5b10212b1e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 385, 'output_tokens': 108, 'total_tokens': 493}
JUST AI TOOL CALLS
[{'name': 'context', 'args': {'k': 4, 'locale': 'en', 'query': 'bleeding first aid', 'topic_id': 'bleed-control'}, 'id': '9f960820-0553-4df4-9794-3e5b10212b1e', 'type': 'tool_call'}]


In [33]:
for tool_call in ai_msg.tool_calls: #Actually running all the tool calls ai requested in last cell
    tools_dict = {"context": context}
    selected_tool = tools_dict[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg) #Add the tool message to chat history

messages




[SystemMessage(content="You are a first-aid assistant. If additional facts are needed, CALL the `context` tool (do not describe it). After receiving tool output, base your answer on it and end with a 'Sources:' line listing citation titles. If no retrieved context is available, say you don't have enough info.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do if bleeding? What to do if snakebite?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-27T02:48:00.578782Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6079019834, 'load_duration': 75429792, 'prompt_eval_count': 385, 'prompt_eval_duration': 2960362417, 'eval_count': 108, 'eval_duration': 3015826583, 'model_name': 'gpt-oss:20b'}, id='run--6b2de3ab-4c8b-4b5c-a3d2-08fe35457a41-0', tool_calls=[{'name': 'context', 'args': {'k': 4, 'locale': 'en', 'query': 'bleeding first aid', 'topic_id': '

In [34]:
ans = llm_with_tools.invoke(messages)
ans

AIMessage(content='**If you’re dealing with bleeding**\n\n1. **Stay calm and assess the situation** – Check the severity of the bleed.  \n2. **Apply firm direct pressure**  \n   * Use a clean cloth, sterile gauze, or your hand.  \n   * Press straight down; do not lift the cloth to check the wound.  \n3. **Add more layers if needed** – If the first cloth soaks through, place a second one on top without removing the first.  \n4. **Elevate the injured limb** (if it’s a limb) while keeping the pressure.  \n5. **Use a tourniquet only if necessary** – If direct pressure fails and the bleed is from an arm or leg, apply a tourniquet above the wound.  \n6. **Note the time** – Record when you started applying pressure; this helps medical staff.  \n7. **Call for help** – If bleeding is severe, continuous, or you’re unsure, get emergency services.\n\n*Source: “Severe Bleeding Control” – WHO Basic Emergency Care (B.E.C.)*\n\n---\n\n**If you’re dealing with a snakebite**\n\n1. **Keep the victim calm

In [35]:
ans.content

'**If you’re dealing with bleeding**\n\n1. **Stay calm and assess the situation** – Check the severity of the bleed.  \n2. **Apply firm direct pressure**  \n   * Use a clean cloth, sterile gauze, or your hand.  \n   * Press straight down; do not lift the cloth to check the wound.  \n3. **Add more layers if needed** – If the first cloth soaks through, place a second one on top without removing the first.  \n4. **Elevate the injured limb** (if it’s a limb) while keeping the pressure.  \n5. **Use a tourniquet only if necessary** – If direct pressure fails and the bleed is from an arm or leg, apply a tourniquet above the wound.  \n6. **Note the time** – Record when you started applying pressure; this helps medical staff.  \n7. **Call for help** – If bleeding is severe, continuous, or you’re unsure, get emergency services.\n\n*Source: “Severe Bleeding Control” – WHO Basic Emergency Care (B.E.C.)*\n\n---\n\n**If you’re dealing with a snakebite**\n\n1. **Keep the victim calm and still** – Mov

### :)